In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tokenizer-test-string/regex_test_string.txt
/kaggle/input/shakesphere-book/shakesphere_book.txt


# **BPE Tokenizer**

In [3]:
!pip install uv
!uv venv bpe-tokenizer
!source /kaggle/working/bpe-tokenizer/bin/activate
!uv pip install huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.4/18.4 MB 60.2 MB/s eta 0:00:00:00:0100:01
Using CPython 3.11.13 interpreter at: /usr/bin/python3
Creating virtual environment at: bpe-tokenizer
Activate with: source bpe-tokenizer/bin/activate
Using Python 3.11.13 environment at: /usr
Resolved 1 package in 126ms                                          
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)--------------     0 B/2.40 KiB            
Prepared 1 package in 26ms                                                   
Installed 1 package in 3ms                                  
 + huggingface==0.0.1


In [4]:
try:
    with open("/kaggle/input/shakesphere-book/shakesphere_book.txt", 'r', encoding="utf-8") as f:
        text_content = f.read()
    
        print("File read successful!")
except Exception as e:
    print(e)

File read successful!


In [5]:
print(text_content[:500])

The Project Gutenberg eBook of The Adventures of Sherlock Holmes
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this ebook or online
at www.gutenberg.org. If you are not located in the United States,
you will have to check the laws of the country where you are located
before using


In [6]:
tokens = text_content.encode("utf-8")
tokens = list(map(int, tokens))
print("=====================")
print("corpus length", len(text_content))
print(tokens[:10])
print("number of tokens", len(tokens))

corpus length 581563
[84, 104, 101, 32, 80, 114, 111, 106, 101, 99]
number of tokens 595338


In [7]:
# to get most frequent occuring pair
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(sorted(((v,k) for k, v in stats.items()), reverse=True))

[(17058, (101, 32)), (13317, (32, 116)), (12093, (116, 104)), (11706, (104, 101)), (10339, (32, 97)), (9778, (100, 32)), (9484, (116, 32)), (8553, (115, 32)), (7834, (105, 110)), (7362, (101, 114)), (6964, (32, 119)), (6876, (32, 104)), (6800, (226, 128)), (6787, (44, 32)), (6614, (97, 110)), (6397, (110, 32)), (6387, (32, 115)), (5776, (114, 101)), (5775, (104, 97)), (5748, (111, 117)), (5448, (32, 111)), (5390, (32, 105)), (5180, (110, 100)), (5071, (97, 116)), (4862, (121, 32)), (4602, (114, 32)), (4579, (111, 110)), (4483, (101, 100)), (4380, (101, 110)), (4264, (111, 32)), (4211, (32, 109)), (4155, (104, 105)), (4134, (105, 115)), (3992, (116, 111)), (3983, (46, 32)), (3829, (105, 116)), (3812, (97, 115)), (3763, (32, 98)), (3592, (110, 103)), (3560, (118, 101)), (3521, (32, 99)), (3501, (101, 115)), (3405, (111, 114)), (3355, (32, 102)), (3236, (97, 114)), (3233, (109, 101)), (3214, (116, 101)), (3205, (115, 116)), (3106, (102, 32)), (2994, (115, 101)), (2986, (111, 102)), (2986,

In [8]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [9]:
# function to merge the top pair to reduce number of tokens
def merge(ids, pair, idx):
    newids=[]
    i=0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [10]:
vocab_size = 500
num_merges = vocab_size - 256
ids = list(tokens)

In [11]:
merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"Merging {pair} into a new token {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

Merging (101, 32) into a new token 256
Merging (116, 104) into a new token 257
Merging (100, 32) into a new token 258
Merging (116, 32) into a new token 259
Merging (115, 32) into a new token 260
Merging (105, 110) into a new token 261
Merging (101, 114) into a new token 262
Merging (226, 128) into a new token 263
Merging (44, 32) into a new token 264
Merging (97, 110) into a new token 265
Merging (111, 117) into a new token 266
Merging (32, 257) into a new token 267
Merging (111, 110) into a new token 268
Merging (121, 32) into a new token 269
Merging (101, 110) into a new token 270
Merging (46, 32) into a new token 271
Merging (116, 111) into a new token 272
Merging (104, 97) into a new token 273
Merging (114, 101) into a new token 274
Merging (111, 114) into a new token 275
Merging (111, 102) into a new token 276
Merging (101, 258) into a new token 277
Merging (267, 256) into a new token 278
Merging (104, 105) into a new token 279
Merging (261, 103) into a new token 280
Merging (97,

> # **Building Decoder pass of Naive BPE tokenizer.**
> ### &emsp; given a token(int representation) return back the string. 

In [12]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    """given ids(integer/list of integers), return python string(associated context)"""
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace") # replace is a standard practice if a token is unknown which returns '@'
    return text



In [13]:
print(decode([97]))
print(decode([128]))

a
�


# Enoder Pass of BPE tokenizer
> ### &emsp; given a python string(s) return token ids

In [14]:
def encode(text):
    """Give a string, return list of tokens"""
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

In [15]:
print(decode(encode("Game of Thrones!")))

Game of Thrones!


In [16]:
valtext = "Many common characters, including numerals, punctuation, and other symbols, are unified within the standard and are not treated as specific to any given writing system. Unicode encodes thousands of emoji, with the continued development thereof conducted by the Consortium as a part of the standard.[4] Moreover, the widespread adoption of Unicode was in large part responsible for the initial popularization of emoji outside of Japan. Unicode is ultimately capable of encoding more than 1.1 million characters."
valtext2 = decode(encode(valtext))
print(valtext2 == valtext)

True


# Replication of GPT tokenizer implementation

In [17]:
!uv pip install regex

Using Python 3.11.13 environment at: /usr
Audited 1 package in 198ms


In [18]:
import regex as re

In [19]:
try:
    with open("/kaggle/input/tokenizer-test-string/regex_test_string.txt", 'r', encoding="utf-8") as f:
        test_string = f.read()
        print("Testing string successfully read!")
except Exception as e:
    print(e)

Testing string successfully read!


## Explanation of pattern of gpt2pat:
1. **|** : represents **"or"**
2. **?** : represents **"optional"**, meaning preceding character could be optional.
3. **+** : **Gready Quantifier** represents **"1 or more occurences**.
4. **\s** : represents **"1 or more whitespace**.
5. **\p{L}** : represents **unicode letters**.
6. **\p{N}** : represents **unicode numbers**.
7. **'s|'t|'re|'ve|'m|'ll|'d** : matches 's or 't or 're and so on.
8. **" ?[\p{L}]+| ?[\p{N}]+ "** : matches optional whitespace with set of letters and numbers.
9. **" ?[^\s\p{L}\p{N}]+"** : matches negation. in this case characters that are not space, letters, numbers.
10. **"\s+(?!\S)|\s+"** : matches multiple occurences of whitespace and any trailing whitespace.

In [20]:
gpt2pat = re.compile(
    r"""'s|'t|'re|'ve|'m|'ll|'d| ?[\p{L}]+| ?[\p{N}]+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
)

In [21]:
%%time
gpt2pat_lst = re.findall(gpt2pat, test_string)

CPU times: user 1.23 ms, sys: 7 µs, total: 1.24 ms
Wall time: 2.6 ms


In [22]:
print(len(gpt2pat_lst))
print(gpt2pat_lst)

454
['Hello', ' there', '!', ' I', "'m", ' testing', ' contractions', ' like', ' don', "'t", ',', ' won', "'t", ',', ' they', "'re", ',', ' we', "'ve", ',', ' I', "'ll", ',', ' and', ' he', "'d", '.', ' Let', "'s", ' see', ' how', ' it', ' handles', ' numbers', ':', ' 123', ',', ' 4', '.', '56', ',', ' -', '789', ',', ' 1', ',', '000', ',', '000', ',', ' and', ' 3', '.', '14159', '.', ' What', ' about', ' special', ' characters', '?', ' @#$%^&*()+={}[]|;:,.<>?/~`!', ' \n', '\n', 'Mixed', ' cases', ':', ' CamelCase', ',', ' snake', '_', 'case', ',', ' kebab', '-', 'case', ',', ' PascalCase', ',', ' UPPER', '_', 'CASE', ',', ' lower', '_', 'case', '.', '\n', '\n', 'Unicode', ' characters', ':', ' café', ',', ' naïve', ',', ' résumé', ',', ' Москва', ',', ' 北京', ',', ' 東京', ',', ' français', ',', ' español', ',', ' português', ',', ' 中文测试', '.', '\n', '\n', 'Punctuation', ' combinations', ':', ' "', 'Hello', ',"', ' she', ' said', '.', ' It', "'s", ' a', ' test', '...', ' really', '?!', '

In [23]:
# GPT 2 refactored code for faster execution
gpt2pat_refactored = re.compile(
    r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}++| ?\p{N}++| ?[^\s\p{L}\p{N}]++|\s++$|\s+(?!\S)|\s"""
)

In [24]:
%%time
gpt2pat_refactored_lst = re.findall(gpt2pat_refactored, test_string)

CPU times: user 565 µs, sys: 992 µs, total: 1.56 ms
Wall time: 1.01 ms


In [25]:
print(len(gpt2pat_refactored_lst))
print(gpt2pat_refactored_lst)

454
['Hello', ' there', '!', ' I', "'m", ' testing', ' contractions', ' like', ' don', "'t", ',', ' won', "'t", ',', ' they', "'re", ',', ' we', "'ve", ',', ' I', "'ll", ',', ' and', ' he', "'d", '.', ' Let', "'s", ' see', ' how', ' it', ' handles', ' numbers', ':', ' 123', ',', ' 4', '.', '56', ',', ' -', '789', ',', ' 1', ',', '000', ',', '000', ',', ' and', ' 3', '.', '14159', '.', ' What', ' about', ' special', ' characters', '?', ' @#$%^&*()+={}[]|;:,.<>?/~`!', ' \n', '\n', 'Mixed', ' cases', ':', ' CamelCase', ',', ' snake', '_', 'case', ',', ' kebab', '-', 'case', ',', ' PascalCase', ',', ' UPPER', '_', 'CASE', ',', ' lower', '_', 'case', '.', '\n', '\n', 'Unicode', ' characters', ':', ' café', ',', ' naïve', ',', ' résumé', ',', ' Москва', ',', ' 北京', ',', ' 東京', ',', ' français', ',', ' español', ',', ' português', ',', ' 中文测试', '.', '\n', '\n', 'Punctuation', ' combinations', ':', ' "', 'Hello', ',"', ' she', ' said', '.', ' It', "'s", ' a', ' test', '...', ' really', '?!', '

In [26]:
# GPT 4 pattern 
# changes: 
# 1. case sensitive matching represented by i.
# 2. fixed length for number i.e 3 represented by {1, 3}.
gpt4pat = re.compile(
    r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}++|\p{N}{1,3}+| ?[^\s\p{L}\p{N}]++[\r\n]*+|\s++$|\s*[\r\n]|\s+(?!\S)|\s"""
)

In [27]:
%%time
gpt4pat_lst = re.findall(gpt4pat, test_string)

CPU times: user 194 µs, sys: 873 µs, total: 1.07 ms
Wall time: 707 µs


In [28]:
print(len(gpt4pat_lst))
print(gpt4pat_lst)

437
['Hello', ' there', '!', ' I', "'m", ' testing', ' contractions', ' like', ' don', "'t", ',', ' won', "'t", ',', ' they', "'re", ',', ' we', "'ve", ',', ' I', "'ll", ',', ' and', ' he', "'d", '.', ' Let', "'s", ' see', ' how', ' it', ' handles', ' numbers', ':', ' ', '123', ',', ' ', '4', '.', '56', ',', ' -', '789', ',', ' ', '1', ',', '000', ',', '000', ',', ' and', ' ', '3', '.', '141', '59', '.', ' What', ' about', ' special', ' characters', '?', ' @#$%^&*()+={}[]|;:,.<>?/~`!', ' \n\n', 'Mixed', ' cases', ':', ' CamelCase', ',', ' snake', '_case', ',', ' kebab', '-case', ',', ' PascalCase', ',', ' UPPER', '_CASE', ',', ' lower', '_case', '.\n\n', 'Unicode', ' characters', ':', ' café', ',', ' naïve', ',', ' résumé', ',', ' Москва', ',', ' 北京', ',', ' 東京', ',', ' français', ',', ' español', ',', ' português', ',', ' 中文测试', '.\n\n', 'Punctuation', ' combinations', ':', ' "', 'Hello', ',"', ' she', ' said', '.', ' It', "'s", ' a', ' test', '...', ' really', '?!', ' (', 'Is', ' thi

In [29]:
# GPT 4o pattern
gpt4opat_str = "|".join(
    [
        r"""[^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]*[\p{Ll}\p{Lm}\p{Lo}\p{M}]+(?i:'s|'t|'re|'ve|'m|'ll|'d)?""",
        r"""[^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]+[\p{Ll}\p{Lm}\p{Lo}\p{M}]*(?i:'s|'t|'re|'ve|'m|'ll|'d)?""",
        r"""\p{N}{1,3}""",
        r""" ?[^\s\p{L}\p{N}]+[\r\n/]*""",
        r"""\s*[\r\n]+""",
        r"""\s+(?!\S)""",
        r"""\s+""",
    ]
)

gpt4opat = re.compile(gpt4opat_str)

In [30]:
%%time
gpt4opat_lst = re.findall(gpt4opat, test_string)

CPU times: user 1.35 ms, sys: 8 µs, total: 1.36 ms
Wall time: 870 µs


In [31]:
print(len(gpt4opat_lst))
print(gpt4opat_lst)

415
['Hello', ' there', '!', " I'm", ' testing', ' contractions', ' like', " don't", ',', " won't", ',', " they're", ',', " we've", ',', " I'll", ',', ' and', " he'd", '.', " Let's", ' see', ' how', ' it', ' handles', ' numbers', ':', ' ', '123', ',', ' ', '4', '.', '56', ',', ' -', '789', ',', ' ', '1', ',', '000', ',', '000', ',', ' and', ' ', '3', '.', '141', '59', '.', ' What', ' about', ' special', ' characters', '?', ' @#$%^&*()+={}[]|;:,.<>?/~`!', ' \n\n', 'Mixed', ' cases', ':', ' Camel', 'Case', ',', ' snake', '_case', ',', ' kebab', '-case', ',', ' Pascal', 'Case', ',', ' UPPER', '_CASE', ',', ' lower', '_case', '.\n\n', 'Unicode', ' characters', ':', ' café', ',', ' naïve', ',', ' résumé', ',', ' Москва', ',', ' 北京', ',', ' 東京', ',', ' français', ',', ' español', ',', ' português', ',', ' 中文测试', '.\n\n', 'Punctuation', ' combinations', ':', ' "', 'Hello', ',"', ' she', ' said', '.', " It's", ' a', ' test', '...', ' really', '?!', ' (', 'Is', ' this', ' working', '?)', ' [', 

# Training BPE Tokenizer

In [125]:
import unicodedata
from pathlib import Path

In [126]:
# Helper Functions

def get_pair_counts(ids, counts=None):
    """Given a list of token ids return their count."""
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """Replacing consecutive pairs with new index."""
    newids = []
    i = 0
    while i < len(ids):
        """if not at last position and the pair matches replace."""
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1

    return newids

def replace_control_characters(s: str) -> str:
    """Replacing charachters such as \n, \t etc. To prevent distortion of output"""
    chars = []
    for ch in s:
        if unicodedata.category(ch)[0] != "C":
            chars.append(ch)
        else:
            chars:append(f"\\u{ord(ch):04x}")

    return "".join(chars)

def render_token(t: bytes) -> str:
    s = t.decode("utf-8", errors="replace")
    s = replace_control_characters(s)
    return s

In [127]:
# Base Tokenizer class
class Tokenizer:
    def __init__(self):
        # default: vocab size 256
        self.merges = {}
        self.pattern = ""
        self.special_tokens = {}
        self.vocab = self._build_vocab()

    def train(self, text, vocab_size, verbose=False):
        raise NotImplementedError

    def encode(self, text):
        raise NotImplementedError

    def decode(self, ids):
        raise NotImplementedError

    def _build_vocab(self):
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for (p0, p1), idx in self.merges.items():
            vocab[idx] = vocab[p0] + vocab[p1]

        for special, idx in self.special_tokens.items():
            vocab[idx] = special.encode("utf-8")
        return vocab

    def save(self, file_prefix):
        """
        Saves two files: file_prefix.vocab and file_prefix.model
        This is inspired (but not equivalent to!) sentencepiece's model saving:
        - model file is the critical one, intended for load()
        - vocab file is just a pretty printed version for human inspection only
        """
        # write the model: to be used in load() later
        model_file = file_prefix + ".model"
        with open(model_file, 'w') as f:
            # write the version, pattern and merges, that's all that's needed
            f.write("minbpe v1\n")
            f.write(f"{self.pattern}\n")
            # write the special tokens, first the number of them, then each one
            f.write(f"{len(self.special_tokens)}\n")
            for special, idx in self.special_tokens.items():
                f.write(f"{special} {idx}\n")
            # the merges dict
            for idx1, idx2 in self.merges:
                f.write(f"{idx1} {idx2}\n")
        # write the vocab: for the human to look at
        vocab_file = file_prefix + ".vocab"
        inverted_merges = {idx: pair for pair, idx in self.merges.items()}
        with open(vocab_file, "w", encoding="utf-8") as f:
            for idx, token in self.vocab.items():
                # note: many tokens may be partial utf-8 sequences
                # and cannot be decoded into valid strings. Here we're using
                # errors='replace' to replace them with the replacement char �.
                # this also means that we couldn't possibly use .vocab in load()
                # because decoding in this way is a lossy operation!
                s = render_token(token)
                # find the children of this token, if any
                if idx in inverted_merges:
                    # if this token has children, render it nicely as a merge
                    idx0, idx1 = inverted_merges[idx]
                    s0 = render_token(self.vocab[idx0])
                    s1 = render_token(self.vocab[idx1])
                    f.write(f"[{s0}][{s1}] -> [{s}] {idx}\n")
                else:
                    # otherwise this is leaf token, just print it
                    # (this should just be the first 256 tokens, the bytes)
                    f.write(f"[{s}] {idx}\n")

    def load(self, model_file):
        """Inverse of save() but only for the model file"""
        assert model_file.endswith(".model")
        # read the model file
        merges = {}
        special_tokens = {}
        idx = 256
        with open(model_file, 'r', encoding="utf-8") as f:
            # read the version
            version = f.readline().strip()
            assert version == "minbpe v1"
            # read the pattern
            self.pattern = f.readline().strip()
            # read the special tokens
            num_special = int(f.readline().strip())
            for _ in range(num_special):
                special, special_idx = f.readline().strip().split()
                special_tokens[special] = int(special_idx)
            # read the merges
            for line in f:
                idx1, idx2 = map(int, line.split())
                merges[(idx1, idx2)] = idx
                idx += 1
        self.merges = merges
        self.special_tokens = special_tokens
        self.vocab = self._build_vocab()

In [128]:
gpt4opat_str = "|".join(
    [
        r"""[^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]*[\p{Ll}\p{Lm}\p{Lo}\p{M}]+(?i:'s|'t|'re|'ve|'m|'ll|'d)?""",
        r"""[^\r\n\p{L}\p{N}]?[\p{Lu}\p{Lt}\p{Lm}\p{Lo}\p{M}]+[\p{Ll}\p{Lm}\p{Lo}\p{M}]*(?i:'s|'t|'re|'ve|'m|'ll|'d)?""",
        r"""\p{N}{1,3}""",
        r""" ?[^\s\p{L}\p{N}]+[\r\n/]*""",
        r"""\s*[\r\n]+""",
        r"""\s+(?!\S)""",
        r"""\s+""",
    ]
)

gpt4opat = re.compile(gpt4opat_str)
gpt4pat = re.compile(
    r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}++|\p{N}{1,3}+| ?[^\s\p{L}\p{N}]++[\r\n]*+|\s++$|\s*[\r\n]|\s+(?!\S)|\s"""
)

In [129]:
class SampleGPT4oTokenizer(Tokenizer):
    def __init__(self, pattern=None):
        """
        - pattern : to override default gpt4o regex pattern
        - special token : additional special token. example <|endoftext|> : 199999, <|endofprompt|>: 200018
        """
        super().__init__()
        self.pattern = gpt4opat if pattern is None else pattern
        self.compiled_pattern = re.compile(self.pattern)
        self.special_tokens = {}
        self.inverse_special_tokens = {}

    def train(self, text, vocab_size, verbose=False):
        assert vocab_size >= 256
        num_merges = vocab_size - 256

        # split the text up into text chunks
        text_chunks = re.findall(self.compiled_pattern, text)

        # input text preprocessing
        ids = [list(ch.encode("utf-8")) for ch in text_chunks]

        # iteratively merge the most common occuring pairs to create new tokens
        merges = {}
        vocab = {idx: bytes([idx]) for idx in range(256)}
        for i in range(num_merges):
            stats = {}
            for chunk_ids in ids:
                get_pair_counts(chunk_ids, stats)

            pair = max(stats, key=stats.get)

            idx = 256 + i

            ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]

            # save merges
            merges[pair] = idx
            vocab[idx] = vocab[pair[0]] + vocab[pair[1]]

            # print
            if verbose:
                print(f"merge {i+1}/{num_merges}: {pair} -> {idx} ({vocab[idx]}) had {stats[pair]} occurences")

        self.merges = merges
        self.vocab = vocab

    def register_special_tokens(self, special_tokens):
        # special_tokens is a dictionary of str -> int
        # example : {"<|endoftext|>" : 199999}
        self.special_tokens = special_tokens
        self.inverse_special_tokens = {v: k for k, v in special_tokens.items()}

    def decode(self, ids):
        """Given list of ids, return python string"""
        part_bytes = []
        for idx in ids:
            if idx in self.vocab:
                part_bytes.append(self.vocab[idx])
            elif idx in self.inverse_special_tokens:
                part_bytes.append(self.inverse_special_tokens[idx].encode("utf-8"))
            else:
                raise ValueError(f"Invalid token id: {idx}")

        text_bytes = b"".join(part_bytes)
        text = text_bytes.decode("utf-8", errors="replace")
        return text

    def _encode_chunk(self, text_bytes):
        """Given list of string, return list of ids"""
        # convert all bytes to integers in range 0.255
        ids = list(text_bytes)
        while len(ids) >= 2:
            # find the pair with the lowest merge index
            stats = get_pair_counts(ids)
            pair = min(stats, key=lambda p: self.merges.get(p, float("inf")))
            if pair not in self.merges:
                break

            idx = self.merges[pair]
            ids = merge(ids, pair, idx)
        return ids

    def encode_ordinary(self, text):
        """Encoding that ignores any special tokens."""
        # split text into chunks of text by categories defined in regex pattern
        text_chunks = re.findall(self.compiled_pattern, text)
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for chunk in text_chunks:
            chunk_bytes = chunk.encode("utf-8") # raw bytes
            chunk_ids = self._encode_chunk(chunk_bytes)
            ids.extend(chunk_ids)
        return ids

    def encode(self, text, allowed_special="none_raise"):
        """
        Unlike encode_ordinary, this function handles special tokens.
        allowed_special: can be "all"|"none"|"none_raise" or a custom set of special tokens
        if none_raise, then an error is raised if any special token is encountered in text
        this is the default tiktoken behavior right now as well
        any other behavior is either annoying, or a major footgun
        """
        # decode the user desire w.r.t. handling of special tokens
        special = None
        if allowed_special == "all":
            special = self.special_tokens
        elif allowed_special == "none":
            special = {}
        elif allowed_special == "none_raise":
            special = {}
            assert all(token not in text for token in self.special_tokens)
        elif isinstance(allowed_special, set):
            special = {k: v for k, v in self.special_tokens.items() if k in allowed_special}
        else:
            raise ValueError(f"allowed_special={allowed_special} not understood")
        if not special:
            # shortcut: if no special tokens, just use the ordinary encoding
            return self.encode_ordinary(text)
        # otherwise, we have to be careful with potential special tokens in text
        # we handle special tokens by splitting the text
        # based on the occurrence of any exact match with any of the special tokens
        # we can use re.split for this. note that surrounding the pattern with ()
        # makes it into a capturing group, so the special tokens will be included
        special_pattern = "(" + "|".join(re.escape(k) for k in special) + ")"
        special_chunks = re.split(special_pattern, text)
        # now all the special characters are separated from the rest of the text
        # all chunks of text are encoded separately, then results are joined
        ids = []
        for part in special_chunks:
            if part in special:
                # this is a special token, encode it separately as a special case
                ids.append(special[part])
            else:
                # this is an ordinary sequence, encode it normally
                ids.extend(self.encode_ordinary(part))
        return ids

In [130]:
import os
import time

tokenizer_classes_to_train = [
    (SampleGPT4oTokenizer, 'regex'),
]

t0 = time.time()
for TokenizerClass, name in tokenizer_classes_to_train:
    print(f"--- Training {name} tokenizer ---")
    tokenizer = TokenizerClass()
    tokenizer.train(text_content, 1128, verbose=True)
    file_path = os.path.join("models", name)
    tokenizer.save(file_path)
t1 = time.time()
print(f"Training took {t1 - t0:.2f} seconds")

--- Training regex tokenizer ---
merge 1/872: (32, 116) -> 256 (b' t') had 13317 occurences
merge 2/872: (104, 101) -> 257 (b'he') had 11706 occurences
merge 3/872: (32, 97) -> 258 (b' a') had 10339 occurences
merge 4/872: (105, 110) -> 259 (b'in') had 7834 occurences
merge 5/872: (32, 119) -> 260 (b' w') had 6964 occurences
merge 6/872: (226, 128) -> 261 (b'\xe2\x80') had 6800 occurences
merge 7/872: (32, 115) -> 262 (b' s') had 6387 occurences
merge 8/872: (256, 257) -> 263 (b' the') had 6088 occurences
merge 9/872: (114, 101) -> 264 (b're') had 5776 occurences
merge 10/872: (104, 97) -> 265 (b'ha') had 5775 occurences
merge 11/872: (111, 117) -> 266 (b'ou') had 5748 occurences
merge 12/872: (32, 111) -> 267 (b' o') had 4889 occurences
merge 13/872: (110, 100) -> 268 (b'nd') had 4666 occurences
merge 14/872: (101, 114) -> 269 (b'er') had 4517 occurences
merge 15/872: (32, 109) -> 270 (b' m') had 4211 occurences
merge 16/872: (105, 115) -> 271 (b'is') had 4134 occurences
merge 17/872:

In [131]:
tokenizer.encode("Hello World")

[72, 442, 111, 418, 287, 321]

In [132]:
tokenizer.decode([72, 442, 111, 418, 287, 321])

'Hello World'

In [133]:
try:
    gpt4o_saved_tokenizer = tokenizer.load("/kaggle/working/models/regex.model")
    print("loaded custom trained gpt4o tokenizer!")
except Exception as e:
    print(e)

loaded custom trained gpt4o tokenizer!


In [134]:
gpt4o_saved_tokenizer.encode("Hi it's Raymond Reddington!")

AttributeError: 'NoneType' object has no attribute 'encode'

In [ ]:
gpt